In [1]:
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

In [2]:
X = np.array([8,4,6,2,7])
H = np.array([1,2])

In [3]:
#valid padding
H_valid = np.matrix([[2,1,0,0,0],
                     [0,2,1,0,0],
                     [0,0,2,1,0], 
                     [0,0,0,2,1]])
conv_valid = np.matmul(H_valid, X)

#full convolution
X_full = np.transpose(np.array([0,8,4,6,2,7,0]))
H_full = np.matrix([[2,1,0,0,0,0,0],
                    [0,2,1,0,0,0,0],
                    [0,0,2,1,0,0,0], 
                    [0,0,0,2,1,0,0],
                    [0,0,0,0,2,1,0],
                    [0,0,0,0,0,2,1]])
conv_full = np.matmul(H_full, X_full)

#same padding
X_same = np.transpose(np.array([0,8,4,6,2,7]))
H_same = np.matrix([[2,1,0,0,0,0],
                    [0,2,1,0,0,0],
                    [0,0,2,1,0,0], 
                    [0,0,0,2,1,0],
                    [0,0,0,0,2,1]])
conv_same = np.matmul(H_same, X_same)

print("valid padding result: \t{} \nfull padding result: \t{} \nsame padding result: \t{}"
       .format(conv_valid, conv_full, conv_same))

valid padding result: 	[[20 14 14 11]] 
full padding result: 	[[ 8 20 14 14 11 14]] 
same padding result: 	[[ 8 20 14 14 11]]


In [4]:
def conv1d(x, h, padding='valid'):
    print("X: {}\nH: {}".format(x, h))
    x = np.array(x)
    if padding=='same':
        for i in range(len(h)-1):
            x = np.insert(x, 0, 0)
    elif padding=='full':
        for i in range(len(h)-1):
            x = np.insert(x, 0, 0)
            x = np.insert(x, len(x), 0)

    """y = []
    for i in range(len(x)-len(h)+1):
        item = 0
        for j in range(len(h)):
            item += x[i+j]*h[len(h)-j-1]
        y.append(item)"""
        
    y = [sum([x[i+j]*h[len(h)-j-1] for j in range(len(h))]) \
         for i in range(len(x) - len(h) + 1)]
    
    #print("ans: {}".format(scipy.signal.convolve(x, h, mode=padding)))
        
    return np.array(y)

#conv1d(X,H, padding='full')
#conv1d([2,4,6,3,5,1],[4,2,9], padding='valid')
conv1d(np.random.randint(10, size=10),np.random.randint(10, size=4), padding='full')

X: [7 8 5 0 3 1 8 5 3 6]
H: [2 4 1 8]


array([14, 44, 49, 84, 75, 54, 23, 67, 42, 93, 67, 30, 48])

In [5]:
def corr1d(x, h, padding='valid'):
    print("X: {}\nH: {}".format(x, h))
    x = np.array(x)
    if padding=='same':
        for i in range(len(h)-1):
            x = np.insert(x, 0, 0)
    elif padding=='full':
        for i in range(len(h)-1):
            x = np.insert(x, 0, 0)
            x = np.insert(x, len(x), 0)
        
    y = [sum([x[i+j]*h[j] for j in range(len(h))]) \
         for i in range(len(x) - len(h) + 1)]
    
    return np.array(y)

corr1d(X,H, padding='valid')

X: [8 4 6 2 7]
H: [1 2]


array([16, 16, 10, 16])

In [6]:
#Prove associative
G = [1,5,9]
print(conv1d(X, conv1d(G,H, padding='full'), padding='full'))
print(conv1d(conv1d(X,H, padding='full'), G, padding='full'))

X: [1, 5, 9]
H: [1 2]
X: [8 4 6 2 7]
H: [ 1  7 19 18]
[  8  60 186 264 207 195 169 126]
X: [8 4 6 2 7]
H: [1 2]
X: [ 8 20 14 14 11 14]
H: [1, 5, 9]
[  8  60 186 264 207 195 169 126]


In [7]:
#Prove communicative
print(conv1d(X, H, padding='full'))
print(conv1d(H, X, padding='full'))

X: [8 4 6 2 7]
H: [1 2]
[ 8 20 14 14 11 14]
X: [1 2]
H: [8 4 6 2 7]
[ 8 20 14 14 11 14]


In [8]:
def conv2d(x, h, padding='valid'):
    print("X: {}\nH: {}".format(x, h))
    x, h = np.array(x), np.array(h)
    if padding=='same':
        for j in range(len(h[0]) - 1):
            x = np.insert(x, 0, 0, axis=1)
        for i in range(len(h) - 1):
            x = np.insert(x, 0, 0, axis=0)
        
    elif padding=='full':
        for j in range(len(h[0]) - 1):
            x = np.insert(x, 0, 0, axis=1)
            x = np.insert(x, len(x[0]), 0, axis=1)
        for i in range(len(h) - 1):
            x = np.insert(x, 0, 0, axis=0)
            x = np.insert(x, len(x), 0, axis=0)

    #print(x)

    x_h, x_w = x.shape[0], x.shape[1]
    h_h, h_w = h.shape[0], h.shape[1]
    
    y = []
    for i in range(x_h - h_h + 1):
        y_sub = []
        for j in range(x_w - h_w + 1):
            new = 0
            for ii in range(h_h):
                for jj in range(h_w):
                    new +=x[i+ii][j+jj] * h[h_h-ii-1][h_w-jj-1]
            y_sub.append(new)
        y.append(y_sub)

    #print("ans: {}".format(scipy.signal.convolve(x, h, mode=padding)))
        
    return np.array(y)

X2 = [[4,6,2,3],
      [3,7,8,9],
      [1,0,2,4]]
H2 = [[2,5,4], 
      [1,6,3]]
conv2d(X2, H2, padding='full')
#conv1d([2,4,6,3,5,1],[4,2,9], padding='valid')
#conv1d(np.random.randint(10, size=10),np.random.randint(10, size=4), padding='full')

X: [[4, 6, 2, 3], [3, 7, 8, 9], [1, 0, 2, 4]]
H: [[2, 5, 4], [1, 6, 3]]


array([[  8,  32,  50,  40,  23,  12],
       [ 10,  59, 113, 119, 101,  45],
       [  5,  30,  67,  96, 106,  43],
       [  1,   6,   5,  16,  30,  12]])

In [9]:
print(conv2d(X2, H2, padding='full'))
print(conv2d(H2, X2, padding='full'))

X: [[4, 6, 2, 3], [3, 7, 8, 9], [1, 0, 2, 4]]
H: [[2, 5, 4], [1, 6, 3]]
[[  8  32  50  40  23  12]
 [ 10  59 113 119 101  45]
 [  5  30  67  96 106  43]
 [  1   6   5  16  30  12]]
X: [[2, 5, 4], [1, 6, 3]]
H: [[4, 6, 2, 3], [3, 7, 8, 9], [1, 0, 2, 4]]
[[  8  32  50  40  23  12]
 [ 10  59 113 119 101  45]
 [  5  30  67  96 106  43]
 [  1   6   5  16  30  12]]


In [10]:
G2 = [[3,11],
      [6,4]]
print(conv2d(X2, conv2d(G2,H2, padding='full'), padding='full'))
print(conv2d(conv2d(X2,H2, padding='full'), G2, padding='full'))

X: [[3, 11], [6, 4]]
H: [[2, 5, 4], [1, 6, 3]]
X: [[4, 6, 2, 3], [3, 7, 8, 9], [1, 0, 2, 4]]
H: [[  6  37  67  44]
 [ 15  67 119  49]
 [  6  40  42  12]]
[[  24  184  502  670  509  289  132]
 [  78  511 1416 2040 1910 1410  543]
 [  75  539 1445 2191 2456 1969  653]
 [  33  229  603  947 1286 1048  304]
 [   6   40   54  116  244  192   48]]
X: [[4, 6, 2, 3], [3, 7, 8, 9], [1, 0, 2, 4]]
H: [[2, 5, 4], [1, 6, 3]]
X: [[  8  32  50  40  23  12]
 [ 10  59 113 119 101  45]
 [  5  30  67  96 106  43]
 [  1   6   5  16  30  12]]
H: [[3, 11], [6, 4]]
[[  24  184  502  670  509  289  132]
 [  78  511 1416 2040 1910 1410  543]
 [  75  539 1445 2191 2456 1969  653]
 [  33  229  603  947 1286 1048  304]
 [   6   40   54  116  244  192   48]]
